## Initial embedding pipeline
Prerequisites:<br>
DBpedia subset hosted with a SPARQL endpoint, relevant datasets (e.g. metacritic-movies) cleaned in previous notebook.

Purpose:<br>
This pipeline builds knowledge graph embeddings for entities of interest (e.g. movies) and trains a classifier to predict the target variable.

(Hint)
Make sure to select the poetry kernel. If it does not show up, try to reload your editor.

In [2]:
import pandas as pd
import os.path
import pickle
import json
from pathlib import Path
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.graphs import KG
from pyrdf2vec.walkers import RandomWalker
from sklearn.svm import SVC
from evaluation_framework.manager import FrameworkManager
from rdflimeConfig import dbpediaLocation, datasets, load_dataset, split_dataset

### Check datasets

In [3]:
for cfg in datasets:
    dataset, entities = load_dataset(cfg)
    display(dataset.head(3))

,Wikidata_URI15,Movie,Release date,DBpedia_URI,label,id,rating,DBpedia_URI15,YAGO_URI15,DBpedia_URI15_Base32
0,http://www.wikidata.org/entity/Q238245,"4 Months, 3 Weeks and 2 Days",1/23/2008 0:00,"http://dbpedia.org/resource/4_Months,_3_Weeks_...",good,1601,97,"http://dbpedia.org/resource/4_Months,_3_Weeks_...",NaN,NB2HI4B2F4XWIYTQMVSGSYJON5ZGOL3SMVZW65LSMNSS6N...
1,http://www.wikidata.org/entity/Q170035,Ratatouille,6/29/2007 0:00,http://dbpedia.org/resource/Ratatouille_(film),good,1602,96,http://dbpedia.org/resource/Ratatouille_(film),http://yago-knowledge.org/resource/Ratatouille...,NB2HI4B2F4XWIYTQMVSGSYJON5ZGOL3SMVZW65LSMNSS6U...
2,http://www.wikidata.org/entity/Q1752684,Killer of Sheep,3/30/2007 0:00,http://dbpedia.org/resource/Killer_of_Sheep,good,1603,94,http://dbpedia.org/resource/Killer_of_Sheep,http://yago-knowledge.org/resource/Killer_of_S...,NB2HI4B2F4XWIYTQMVSGSYJON5ZGOL3SMVZW65LSMNSS6S...


,Wikidata_URI15,Movie,Release date,DBpedia_URI,label,id,rating,DBpedia_URI15,YAGO_URI15,DBpedia_URI15_Base32
0,http://www.wikidata.org/entity/Q238245,"4 Months, 3 Weeks and 2 Days",1/23/2008 0:00,"http://dbpedia.org/resource/4_Months,_3_Weeks_...",good,1601,97,"http://dbpedia.org/resource/4_Months,_3_Weeks_...",NaN,NB2HI4B2F4XWIYTQMVSGSYJON5ZGOL3SMVZW65LSMNSS6N...
1,http://www.wikidata.org/entity/Q170035,Ratatouille,6/29/2007 0:00,http://dbpedia.org/resource/Ratatouille_(film),good,1602,96,http://dbpedia.org/resource/Ratatouille_(film),http://yago-knowledge.org/resource/Ratatouille...,NB2HI4B2F4XWIYTQMVSGSYJON5ZGOL3SMVZW65LSMNSS6U...
2,http://www.wikidata.org/entity/Q1752684,Killer of Sheep,3/30/2007 0:00,http://dbpedia.org/resource/Killer_of_Sheep,good,1603,94,http://dbpedia.org/resource/Killer_of_Sheep,http://yago-knowledge.org/resource/Killer_of_S...,NB2HI4B2F4XWIYTQMVSGSYJON5ZGOL3SMVZW65LSMNSS6S...


### Build DBpedia embeddings

Train and store a PyRDF2Vec transformer with various parameter settings.

In [14]:
for cfg in datasets:
    for algo in [0,1]: # 0: CBOW, 1: SG
        for vsize in [50, 100, 200]: # Vector size of embeddings
            print(cfg["name"], algo, vsize)

            dataset, entities = load_dataset(cfg)
            datasetLocation = cfg["location"]
            
            if os.path.exists(os.path.join(datasetLocation, f"rdf2vec_transformer_{'sg' if algo else 'cbow'}_{vsize}")):
                print(f"Skipping transformer, as it already exists.")
                continue
            
            dbpedia = KG(dbpediaLocation, skip_verify=False, mul_req=False)
            
            transformer = RDF2VecTransformer(
                Word2Vec(negative=25, sg=algo, vector_size=vsize),
                walkers=[RandomWalker(max_walks=22, max_depth=2, with_reverse=True, n_jobs=8, md5_bytes=None)],
                verbose=1
            )

            walks = transformer.get_walks(dbpedia, entities)
            transformer.fit(walks)
            embeddings, literals = transformer.transform(dbpedia, entities)

            targetPath = os.path.join(datasetLocation, "transformers")
            Path(targetPath).mkdir(parents=True, exist_ok=True)
            transformer.save(os.path.join(targetPath, f"rdf2vec_transformer_{'sg' if algo else 'cbow'}_{vsize}"))

0 50


100%|██████████| 2000/2000 [02:58<00:00, 11.20it/s]


Extracted 967877 walks for 2000 entities (178.8813s)
Fitted 967877 walks (33.0955s)
0 100


100%|██████████| 2000/2000 [03:07<00:00, 10.68it/s]


Extracted 967877 walks for 2000 entities (187.6276s)
Fitted 967877 walks (37.3635s)
0 200


...

Extracted 967877 walks for 2000 entities (209.7185s)
Fitted 967877 walks (89.9783s)
1 100


100%|██████████| 2000/2000 [03:28<00:00,  9.60it/s]


Extracted 967877 walks for 2000 entities (208.8070s)
Fitted 967877 walks (105.5400s)
1 200


100%|██████████| 2000/2000 [03:22<00:00,  9.86it/s]


Extracted 967877 walks for 2000 entities (203.2874s)
Fitted 967877 walks (151.5139s)


### Store embeddings that are compatible to the Evaluation Framework
In a next step, we would like to evaluate the quality of our generated embeddings. However, we first need to convert to a format that is readable by [GEval](https://github.com/mariaangelapellegrino/Evaluation-Framework), the graph embedding framework by Pellegrino et al.

This entails two steps: First, apply the fixes from 1_DBpediaFixes.ipynb in reverse, i.e. "unfix" the IRIs to be compatible with the framework. Second, store the embeddings in the required CSV-like format.

In [15]:
for cfg in datasets:
    datasetLocation = cfg["location"]
    _, entities = load_dataset(cfg)

    with open(os.path.join(datasetLocation, "datasetFixes.json"), "r") as f:
        fixes = json.load(f)

    for algo in ["cbow", "sg"]:
        for vsize in [50, 100, 200]: # Vector size of embeddings

            with open(os.path.join(datasetLocation, "transformers", f"rdf2vec_transformer_{algo}_{vsize}"), "rb") as file:
                transformer: RDF2VecTransformer = pickle.load(file)

            targetPath = os.path.join(datasetLocation, "embeddings")
            Path(targetPath).mkdir(parents=True, exist_ok=True)

            with open(os.path.join(targetPath, f"embeddings_{algo}_{vsize}"), "w") as file:
                for index, embedding in enumerate(transformer._embeddings):

                    # "Unfix" IRI and replace with the version that the Evaluation Framework by Pellegrino et al. understands
                    movie = entities[index]
                    fix = next(filter(lambda f: f["fix"] == movie, fixes), None)
                    if fix: movie = fix["original"]            

                    # Write embedding to file
                    line = f"{movie} {' '.join(map(str,embedding))}\n"
                    file.write(line)

## Evaluation

### Evaluation using Evaluation-Framework by Pellegrino et al.
Load each of our embedding versions and run the classification task on the movie dataset.

In [3]:
evalPath = os.path.join(movieLocation, "embeddings", "evaluation")
Path(evalPath).mkdir(parents=True, exist_ok=True)

for algo in ["cbow", "sg"]:
    for vsize in [50, 100, 200]: # Vector size of embeddings
        print(algo, vsize)

        embeddingPath = os.path.join(movieLocation, "embeddings", f"embeddings_{algo}_{vsize}")
        
        evaluation_manager = FrameworkManager()
        evaluation_manager.evaluate(
            embeddingPath,
            tasks=["Classification"],
            parallel=False,
            debugging_mode=False,
            vector_size=vsize,
            result_directory_path=os.path.join(evalPath, f"geval_result_{algo}_{vsize}")
        )

Start evaluation...
Classification finished
0:01:49
Start evaluation...
Classification finished
0:03:10
Start evaluation...
Classification finished
0:06:05
Start evaluation...
Classification finished
0:02:02
Start evaluation...
Classification finished
0:03:28
Start evaluation...
Classification finished
0:06:44


In [5]:
# Move the results file to the proper location
!mv comparison.csv $evalPath

In [10]:
results = pd.read_csv(os.path.join(evalPath, "comparison.csv"), sep=" ")
tab = pd.DataFrame()

for algo in ["cbow", "sg"]:
    for vsize in [50, 100, 200]:
        r = results[results.test_name.str.contains(f"{algo}_{vsize}")] \
            .groupby("model") \
            .max() \
            .reset_index()

        row = {"strategy": f"{algo}_{vsize}"}
        for m in ["NB", "KNN", "SVM", "C45"]:
            row[m] = round(r[r.model==m].iloc[0]["score_value"]*100, 2)
        tab = pd.concat([tab, pd.DataFrame([row])], ignore_index=True)

tab

,strategy,NB,KNN,SVM,C45
0,cbow_50,75.35,80.64,86.04,67.65
1,cbow_100,77.65,81.15,89.29,68.20
2,cbow_200,76.88,83.17,91.04,68.54
3,sg_50,69.76,72.28,78.82,57.86
4,sg_100,70.90,73.66,83.78,57.04
5,sg_200,71.42,72.53,85.70,57.98


### Learn final classifier on embeddings
Testing with the framework by Pellegrino et al. (see above) reveals that SVC with C=100 delivers high accuracy on the given task (predicting movie quality). We therefore train and store such a classifier for every embedding variant that was trained. 

In [13]:
for cfg in datasets:
    for algo in [0,1]: # 0: CBOW, 1: SG
        for vsize in [50,100,200]: # Vector size of embeddings

            dataset, entities = load_dataset(cfg)
            train, test = split_dataset(dataset, cfg)
            datasetLocation = cfg["location"]
            label_col = cfg["columns"]["label"]
        
            transformerPath = os.path.join(datasetLocation, "transformers", f"rdf2vec_transformer_{'sg' if algo else 'cbow'}_{vsize}")
            transformer = RDF2VecTransformer.load(transformerPath)

            clf = SVC(C=100, probability=True)

            # train-test split as seen in 0_MovieDataSetExploration.ipynb
            train_partition = cfg["train_partition"]
            clf.fit(transformer._embeddings[train_partition[0]:train_partition[1]], train[label_col])

            targetPath = os.path.join(datasetLocation, "classifiers")
            Path(targetPath).mkdir(parents=True, exist_ok=True)
            
            with open(os.path.join(targetPath, f"svc_100_{'sg' if algo else 'cbow'}_{vsize}" ), "wb") as file:
                pickle.dump(clf, file)

## Experiments

In [35]:
walks = transformer._walks

print(f"Number of entities with walks: {len(walks)}")
print(f"Walks per entity: {len(walks[0])}")
print(f"First walk of first entity:")
print(walks[0][0])

# Distance(Matrix, Matrix Reloaded) < Distance(Matrix, The Batman)?
# Can we represent the embeddings in a 2D space for visualization? -> See examples

Number of entities with walks: 2000
Walks per entity: 484
First walk of first entity:
('http://dbpedia.org/resource/Category:Romanian_films_by_genre', 'http://www.w3.org/2004/02/skos/core#broader', 'http://dbpedia.org/resource/Category:Romanian_drama_films', 'http://purl.org/dc/terms/subject', 'http://dbpedia.org/resource/4_Months,_3_Weeks_and_2_Days', 'http://purl.org/dc/terms/subject', 'http://dbpedia.org/resource/Category:European_Film_Awards_winners_(films)', 'http://www.w3.org/2004/02/skos/core#prefLabel', 'European Film Awards winners (films)')


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

transformer = RDF2VecTransformer.load(os.path.join(movieLocation, "transformers", "rdf2vec_transformer_cbow_200"))

# Reduce the dimensions of entity embeddings to represent them in a 2D plane.
X_tsne = TSNE(random_state=42).fit_transform(transformer._embeddings[:])

colors = list(map(lambda e: "#00ff00" if movieFull[movieFull.DBpedia_URI==e].iloc[0].label == "good" else "#ff0000", transformer._entities[:]))
sizes = list(map(lambda e: abs(50-movieFull[movieFull.DBpedia_URI==e].iloc[0].rating)**2, transformer._entities[:]))

# Plot the embeddings of entities in a 2D plane, annotating them.
f = plt.figure(figsize=(200, 80))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=colors, s=sizes)
for x, y, t in zip(X_tsne[:, 0], X_tsne[:, 1], transformer._entities):
    plt.annotate(t.split("/")[-1], (x, y))

# Display the graph with a title, removing the axes for better readability.
plt.title("pyRDF2Vec", fontsize=4)
plt.axis("off")
#plt.show()

f.savefig(os.path.join(movieLocation, "figure.pdf"), bbox_inches='tight')